In [1]:
import numpy as np
import pandas as pd
import catboost as cbt

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/catboost_model'

In [4]:
def gen_xy(sequence):
    """
    Returns
    ---
    x: (num_samples, 5)
    y: (num_samples,) 1-d vec for labels
    """

    x, y = [], []
    for i in range(len(sequence) - 2):
        x.append(sequence[i : i + 2])
        y.append(sequence[i + 1])

    x = np.array(x)
    x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    x = x[:, :5]
    y = np.array(y)

    if len(y.shape) > 1:
        y = y[:, -1]

    return x, y

sequence = pd.read_pickle("../data/data_100000_distr.pkl")[
    ["from_user_id", "to_user_id", "label"]
].values

x, y=gen_xy(sequence)

x.shape
y.shape

(99998, 5)

(99998,)

In [6]:
train_size=0.7
val_size=0.1
cat_feat_index=[0, 1, 2, 3, 4]

split1 = int(len(x) * train_size)
split2 = int(len(sequence) * (train_size + val_size))

x_train, y_train = x[:split1], y[:split1]
x_val, y_val = x[split1:split2], y[split1:split2]
x_test, y_test = x[split2:], y[split2:]

print(f"Trainset:\tx-{x_train.shape}\ty-{y_train.shape}")
print(f"Valset:  \tx-{x_val.shape}  \ty-{y_val.shape}")
print(f"Testset:\tx-{x_test.shape}\ty-{y_test.shape}")

trainset=cbt.Pool(data=x_train, label=y_train, cat_features=cat_feat_index)
valset=cbt.Pool(data=x_val, label=y_val, cat_features=cat_feat_index)
testset=cbt.Pool(data=x_test, label=y_test, cat_features=cat_feat_index)

Trainset:	x-(69998, 5)	y-(69998,)
Valset:  	x-(10002, 5)  	y-(10002,)
Testset:	x-(19998, 5)	y-(19998,)


In [10]:
model=cbt.CatBoostClassifier(iterations=100, learning_rate=0.01, depth=3, loss_function="CrossEntropy", verbose=10)

model.fit(trainset, eval_set=valset)

0:	learn: 0.6920339	test: 0.6920413	best: 0.6920413 (0)	total: 4.37ms	remaining: 433ms
10:	learn: 0.6820454	test: 0.6821232	best: 0.6821232 (10)	total: 42ms	remaining: 340ms
20:	learn: 0.6675022	test: 0.6652223	best: 0.6652223 (20)	total: 85.8ms	remaining: 323ms
30:	learn: 0.6529174	test: 0.6478790	best: 0.6478790 (30)	total: 124ms	remaining: 276ms
40:	learn: 0.6404790	test: 0.6330350	best: 0.6330350 (40)	total: 160ms	remaining: 230ms
50:	learn: 0.6300694	test: 0.6200727	best: 0.6200727 (50)	total: 197ms	remaining: 190ms
60:	learn: 0.6197152	test: 0.6070259	best: 0.6070259 (60)	total: 238ms	remaining: 152ms
70:	learn: 0.6076954	test: 0.5916937	best: 0.5916937 (70)	total: 275ms	remaining: 112ms
80:	learn: 0.5972827	test: 0.5783824	best: 0.5783824 (80)	total: 313ms	remaining: 73.3ms
90:	learn: 0.5882480	test: 0.5665336	best: 0.5665336 (90)	total: 349ms	remaining: 34.5ms
99:	learn: 0.5810359	test: 0.5568742	best: 0.5568742 (99)	total: 382ms	remaining: 0us

bestTest = 0.5568742223
bestIter